# Wave reflection at the interface between two media

This notebook illustrates the reflection of a wave impinging to the interface between two media of different impedances.

It was written to illustrate the eigth exercise of 'TD Ondes'.

Author : Matthieu Hartenstein, contact me here : matthieu.hartenstein@sorbonne-universite.fr

In [5]:
# Basic imports

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

%matplotlib notebook

from IPython.display import HTML



## A bit of theory

Let's assume that an acoustic progressive plane wave 

$$ 
p_I(x,t) = e^{j(k_1 x_1 - \omega t)}
$$

arrives from the left, and impiges at the interface between two media. Let's call these media $1$ (on the left) and $2$ (on the right). Medium $i$ has impedance $Z_i$, density $\rho_i$, speed of sound $c_i$.

At the interface, a wave $p_R$ is reflected into medium $1$, and a wave $p_T$ is transmitted into medium $2$. This gives the following expressions for the pressure in both media

$$
p_1 = e^{j(k_1 x_1 - \omega t)} + r_p e^{-j(k_1 x_1 + \omega t)} \\
p_2 = t_p e^{j(k_2 x_2 - \omega t)}
$$

and similar expressions for the particle velocities

$$
u_1 = \frac{1}{Z_1} (e^{j(k_1 x_1 - \omega t)} - r_p e^{-j(k_1 x_1 + \omega t)}) \\
u_2 = \frac{t_p}{Z_2} e^{j(k_2 x_2 - \omega t)}.
$$

$r_p$ (resp. $t_p$) is called the pressure reflection coefficient.

Using the condition of continuity in terms of pressure and particle velocity at the interface between the media gives the following relations between the reflection and transmission coefficients

$$
1 + r_p = t_p \\
(1 - r_p) \frac{Z_2}{Z_1} = t_p 
$$

Inverting this system of equations allows to obtain the expressions of the reflection and transmission coefficients in terms of the ratio between the media impedances :

$$
r_p = \frac{Z_2 - Z_1}{Z_2 + Z_1} \\
t_p = \frac{2 Z_2}{Z_2 + Z_1}
$$

## 

In [6]:
# Impedances of the media 
Z1 = .1
Z2 = 10

rho = 1

# Frequency 
omega = 2*np.pi

# Dimensions of the media
L = 2
Nx = 10000
x1 = np.linspace(-L, 0, Nx)
x2 = np.linspace(0,L, Nx)
x = np.hstack((x1[np.newaxis,:], x2[np.newaxis,:]))

def comp_pars():
    # Compute speeds of sound
    c1 = Z1/rho
    c2 = Z2/rho
    # Compute pressure reflection and transmission coefficients
    rp = (Z2 - Z1)/(Z2 + Z1)
    tp = 2*Z2/(Z2+Z1)
    return c1, c2, rp, tp

c1, c2, rp, tp = comp_pars()


## Observations

Let's make some observations on the following animations (click on the '>' button after running the cell to start the animations): 

- If $Z_2 = Z_1$ (the impedances match), there is no reflected wave ($r_p = 0$), the wave is fully transmitted ($t_p = 1$).
- If $Z_2 >> Z_1$ (medium $2$ is way harder than medium $1$), $|r_p| = 1$ and the reflected wave is in phase with the incident one. This is called a rigid condition (the particle velocity will be null at the interface). Try $(Z_1, Z_2) = (0.1,10)$.
- If $Z_2 <<Z_1$, there is no transmitted wave ($t_p = 0$), $|r_p = 1|$, and the reflected wave is out of phase with the incident one. This is called a pressure release condition ('condition de surpression nulle', in french), the pressure will be null at the interface. Try $(Z_1, Z_2) = (10,0.1)$.

### Pressure fields

In [7]:
def comp_frames_pressure():

    # Define an animation
    Nframes = 20
    interval = 50

    def draw_pressure(frame_num):
        k1 = omega/c1
        k2 = omega/c2
        t = frame_num/Nframes
        pI = np.exp(1j*(k1*x1 - omega*t)) # incident wave
        pR = rp*np.exp(-1j*(k1*x1 + omega*t))
        pT = tp*np.exp(1j*(k2*x2 - omega*t))
        p = np.hstack((pI + pR, pT))
        line1.set_data((x1, np.real(pI)))
        line2.set_data((x1, np.real(pR)))
        line3.set_data((x2, np.real(pT)))
        line4.set_data((x, np.real(p)))
        return line1, line2, line3, line4

    # Compute the frames
    anim = FuncAnimation(fig_pressure, draw_pressure, frames = Nframes, interval = interval)
    return anim

# Prepare the plot
fig_pressure = plt.figure()
ax1 = fig_pressure.add_subplot(211)
line1, = ax1.plot([])
line2, = ax1.plot([])
line3, = ax1.plot([])
ax2 = fig_pressure.add_subplot(212)
line4, = ax2.plot([])
for ax in [ax1, ax2]:
    ax.set_xlim([-L, L])
    ax.set_ylim([-2.1, 2.1])
    
    ax.set_xlabel('x (m)')
    ax.set_ylabel(r'$real(p)$')
    plt.grid()

ax1.legend(['$p_I$','$p_R$','$p_T$'],loc='center left', bbox_to_anchor=(1, 0.5))
ax1.set_title('Pressure fields \n' + r'$\frac{Z_2}{Z_1} = %f$'%(Z2/Z1))
ax2.set_title('Total pressure')
plt.tight_layout()

anim_pressure = comp_frames_pressure()

HTML(anim_pressure.to_jshtml())

<IPython.core.display.Javascript object>

### Velocity fields


In [8]:
def comp_frames_velocity():

    # Define an animation
    Nframes = 20
    interval = 50

    def draw_velocity(frame_num):
        k1 = omega/c1
        k2 = omega/c2
        t = frame_num/Nframes
        uI = 1/Z1*np.exp(1j*(k1*x1 - omega*t)) # incident wave
        uR = -rp/Z1*np.exp(-1j*(k1*x1 + omega*t))
        uT = tp/Z2*np.exp(1j*(k2*x2 - omega*t))
        u = np.hstack((uI + uR, uT))
        line1.set_data((x1, np.real(uI)))
        line2.set_data((x1, np.real(uR)))
        line3.set_data((x2, np.real(uT)))
        line4.set_data((x, np.real(u)))
        return line1, line2, line3, line4

    # Compute the frames
    anim = FuncAnimation(fig_velocity, draw_velocity, frames = Nframes, interval = interval)
    return anim

# Prepare the plot
fig_velocity = plt.figure()
ax1 = fig_velocity.add_subplot(211)
line1, = ax1.plot([])
line2, = ax1.plot([])
line3, = ax1.plot([])
ax2 = fig_velocity.add_subplot(212)
line4, = ax2.plot([])
for ax in [ax1, ax2]:
    ax.set_xlim([-L, L])
    ax.set_ylim([-2.1/Z1, 2.1/Z1])
    
    ax.set_xlabel('x (m)')
    ax.set_ylabel(r'$real(u)$')
    plt.grid()

ax1.legend(['$u_I$','$u_R$','$u_T$'],loc='center left', bbox_to_anchor=(1, 0.5))
ax1.set_title('Velocity fields \n' + r'$\frac{Z_2}{Z_1} = %f$'%(Z2/Z1))
ax2.set_title('Total velocity')
plt.tight_layout()

anim_velocity = comp_frames_velocity()

HTML(anim_velocity.to_jshtml())

<IPython.core.display.Javascript object>